#**Basics of popular Python libraries for AI**

In [ ]:
# Data processing using Pandas...


import numpy as np
import pandas as pd


data = pd.read_csv("/report.csv", na_values = "/")                                                                           # Loading data from CSV file (with replacement of "/" values to NaN values)...

print(data.head(10))                                                                                                         # Reading first 5 rows...
print(data.tail(10))                                                                                                         # Reading last 10 rows...

print(data["Ispit1"])                                                                                                        # Reading specific column of DataFrame object
print(data["Ispit2"])

print(data.loc[data["Prisustvo"] == 0])                                                                                      # Reading rows with specific characteristic...

data.replace("/", np.nan, inplace = True)                                                                                    # Replacing values (this is only for informative purpose, these values are already replaced)...

print(data.loc[data["Ocjena"] > 7].loc[:, ["Indeks", "UKUPNO", "Ocjena"]])                                                   # Reading specific columns of rows with specific characteristic...

data.dropna(subset = ["Ocjena"], inplace = True)                                                                             # Deleting specific rows of DataFrame object...

temporaryStorage = data.replace(np.nan, -1)
data["Ispit1_final"] = np.maximum(temporaryStorage["Ispit1"], temporaryStorage["Ispit1_popravni"]).replace(-1, np.nan)       # Adding new column (with specific values)...
data["Ispit2_final"] = np.maximum(temporaryStorage["Ispit2"], temporaryStorage["Ispit2_popravni"]).replace(-1, np.nan)

data.drop(columns = ["Ispit1", "Ispit2", "Ispit1_popravni", "Ispit2_popravni"], inplace = True)                              # Deleting specific columns of DataFrame object...

data.to_csv("report-update.csv", sep = ";")                                                                                  # Saving DataFrame object as CSV file...
data.to_pickle("report-update.p")                                                                                            # Saving DataFrame object as Pickle file...                                                            

In [94]:
# Data normalization using Sklearn...


import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


data = pd.read_csv("/report.csv", na_values = "/")
simpleImputerMedian = SimpleImputer(strategy = "median")                                                    # Creating imputers for handling NaN values...
simpleImputerMean = SimpleImputer(strategy = "mean")             
minMaxScaler = MinMaxScaler()                                                                               # Creating MinMax scaler...

data["Ispit1"] = simpleImputerMedian.fit_transform(data["Ispit1"].values.reshape(-1, 1))                    # Replacing NaN values with "median" strategy...
data["Ispit2"] = simpleImputerMedian.fit_transform(data["Ispit2"].values.reshape(-1, 1))
data["Ispit1_popravni"] = simpleImputerMean.fit_transform(data["Ispit1_popravni"].values.reshape(-1, 1))    # Replacing NaN values with "mean" strategy...
data["Ispit2_popravni"] = simpleImputerMean.fit_transform(data["Ispit2_popravni"].values.reshape(-1, 1))

data["Ispit1"] = scale(data["Ispit1"])                                                                      # Normalizing values using Z-score normalization...
data["Ispit2"] = scale(data["Ispit2"])
data["Ispit1_popravni"] = minMaxScaler.fit_transform(data["Ispit1_popravni"].values.reshape(-1, 1))         # Normalizing values using MinMax scaler...
data["Ispit2_popravni"] = minMaxScaler.fit_transform(data["Ispit2_popravni"].values.reshape(-1, 1))       

data.replace(np.nan, 0, inplace = True)                                                                     # Other NaN values will be 0...

grades = data["Ocjena"]
data.drop(columns = ["Ocjena"], inplace = True)                                                             # Deleting column with grades...

gradesNumPyArray = grades.to_numpy()                                                                        # Converting DataFrame object to NumPy array...
dataNumPyArray = data.to_numpy()

xTrain, xTest, yTrain, yTest = train_test_split(dataNumPyArray, gradesNumPyArray, test_size = 0.2)          # Preparing for classification training...

In [ ]:
# Classification of Iris flower using Sklearn...


from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics


iris = load_iris()                                                                                  # Loading dataset...

x = iris.data
y = iris.target
featureNames = iris.feature_names
targetNames = iris.target_names

print(f"Feature names: {featureNames}")
print(f"Target names: {targetNames}")
print(f"Data examples: {x[:5]}")

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.3, random_state = 1)            # Preparing dataset for training and evaluation...

knnClassifier = KNeighborsClassifier(n_neighbors = 3)                                               # Preparing k-Nearest Neighbor cassifier...

knnClassifier.fit(xTrain, yTrain)                                                                   # Training...

yPredicted = knnClassifier.predict(xTest)                                                           # Testing...
accuracy = metrics.accuracy_score(yTest, yPredicted)
print(f"Accuracy: {accuracy}") 

In [29]:
# Deep learning using Keras (MNIST dataset)...


from keras.datasets import mnist                                                                        # Importing MNIST dataset...
from keras import models
from keras import layers
from keras.utils import to_categorical


(xTrain, yTrain), (xTest, yTest) = mnist.load_data()                                                                # Preparing training and testing sets...

print(f"Training set dimensions: {xTrain.shape}")
print(f"Training set number of elements: {xTrain.shape[0]}")
print(f"Training set size (bytes): {xTrain.itemsize * xTrain.size}")

print(f"Testing set dimensions: {xTest.shape}")
print(f"Testing set number of elements: {xTest.shape[0]}")
print(f"Testing set size (bytes): {xTest.itemsize * xTest.size}")

trainImages = xTrain.reshape((xTrain.shape[0], 28 * 28))                                                            # Preparing data for artificial neural network...
trainImages = trainImages.astype("float32") / 255

testImages = xTest.reshape((xTest.shape[0], 28 * 28))              
testImages = testImages.astype("float32") / 255

print(f"Training set size (bytes): {trainImages.itemsize * trainImages.size}")
print(f"Testing set size (bytes): {testImages.itemsize * testImages.size}")

neuralNetworkModel = models.Sequential()                                                                            # Creating neural network model...
neuralNetworkModel.add(layers.Dense(512, activation = "relu", input_shape = (28 * 28,)))                
neuralNetworkModel.add(layers.Dense(10, activation = "softmax"))

neuralNetworkModel.compile(optimizer = "rmsprop", loss = "categorical_crossentropy", metrics =["accuracy"])         # Compiling model...

trainLabels = to_categorical(yTrain)                                                                                # Preparing labels...                  
testLabels = to_categorical(yTest)

neuralNetworkModel.fit(trainImages, trainLabels, epochs = 5, batch_size = 128)                                      # Neural network model training...

testLoss, testAccuracy = neuralNetworkModel.evaluate(testImages, testLabels)                                        # Evaulating model...
print(f"Accuracy (on testing data): {testAccuracy}")

Training set dimensions: (60000, 28, 28)
Training set number of elements: 60000
Training set size (bytes): 47040000
Testing set dimensions: (10000, 28, 28)
Testing set number of elements: 10000
Testing set size (bytes): 7840000
Training set size (bytes): 188160000
Testing set size (bytes): 31360000
Epoch 1/5
469/469 [==============================] - 4s 6ms/step - loss: 0.2674 - accuracy: 0.9212
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 0.1075 - accuracy: 0.9683
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0719 - accuracy: 0.9786
Epoch 4/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0518 - accuracy: 0.9848
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0661 - accuracy: 0.9805
Accuracy (on testing data): 0.9804999828338623
